In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from utils import *

%matplotlib inline

In [2]:
#load the historial data in 2017 and 2018
#here I use data from 2017-02-14-2017-07-01 and 2018-02-01 2018-04-01
aq_old,ob_old,grid_old = load_old_all(start_17='2017-02-14 00:00:00',end_17='2017-06-01 00:00:00',start_18='2018-02-01 00:00:00',end_18='2018-04-01 00:00:00')

#prepare weather data
weather_old_all = pd.concat([ob_old,grid_old]).reset_index()
weather_old_all = clean_weather(weather_old_all)
weather_old_all = weather_old_all.drop(['latitude','longitude'],axis=1)

#add time columns to airQuality data
#levels is the number of nearest weather station to search
#time_column is the name of the time column
aq_old_all = fill_weather_gap(aq_df=aq_old,weather_df=weather_old_all,levels = 5,time_column = 'utc_time')

aq_old_all['timestamp'] = pd.to_datetime(aq_old_all['utc_time'])
aq_old_all['year'] = aq_old_all['timestamp'].dt.year
aq_old_all['month'] = aq_old_all['timestamp'].dt.month
aq_old_all['day'] = aq_old_all['timestamp'].dt.day
aq_old_all['hour'] = aq_old_all['timestamp'].dt.hour

Level 1 #null before 2752
After:  1938
Level 2 #null before 1938
After:  1254
Level 3 #null before 1254
After:  912
Level 4 #null before 912
After:  570


In [3]:
aq_old_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124719 entries, 0 to 923
Data columns (total 21 columns):
utc_time           124719 non-null object
PM2.5              118474 non-null float64
PM10               95007 non-null float64
NO2                119133 non-null float64
CO                 106258 non-null float64
O3                 118450 non-null float64
SO2                119213 non-null float64
longitude          124719 non-null float64
latitude           124719 non-null float64
station_type_id    124719 non-null int64
dist               124719 non-null float64
w_end              124719 non-null object
humidity           124149 non-null float64
pressure           124149 non-null float64
temperature        124149 non-null float64
station_id         124719 non-null object
timestamp          124719 non-null datetime64[ns]
year               124719 non-null int64
month              124719 non-null int64
day                124719 non-null int64
hour               124719 non-null in

In [4]:
#TODO:predict aq loss value
aq_old_all = aq_old_all.fillna(method='ffill')

### Prepare data for feature extraction

In [5]:
aq_2017 = aq_old_all.loc[aq_old_all.year == 2017]
aq_2018 = aq_old_all.loc[aq_old_all.year == 2018]
time_columns = ['year','month','day','hour']
attrs_to_predict = ['PM2.5', 'PM10', 'O3']
timestamp_column_name = 'utc_time'

#result will be numpy array in the format (station_id,station_type_id,time_columns,historical_data-> 24*7*3 * 3,to_be_predicted_values)
data_2017 = get_raw_train_test_data(aq_2017,attrs_to_predict,time_columns,timestamp_column_name,24*7*3)
data_2018 = get_raw_train_test_data(aq_2018,attrs_to_predict,time_columns,timestamp_column_name,24*7*3)

data_2017_2018_raw = np.array(data_2017+data_2018)

In [6]:
#store the file for quick load
np.save('/mnt/disks/bdt/data_2017_2018_before_extract_v1.npy',data_2017_2018_raw)

### Extract features from the raw historical data

In [2]:
data_2017_2018_raw = np.load('/mnt/disks/bdt/data_2017_2018_before_extract_v1.npy')

In [3]:
#get the features and labels from raw historical data
#attr is the attribute we need to predict,it can be 'PM2.5','PM10','O3'
#length is the window size of the number of historical data we used, here we used the previous 3 weeks to predict the value in next 49 hours
X_raw,y_raw = split_features_labels(data = data_2017_2018_raw,attr = 'PM2.5',length = 24*7*3)

In [4]:
np.save('/mnt/disks/bdt/2017_2018_X_v1.npy',X_raw)
np.save('/mnt/disks/bdt/2017_2018_y_v1.npy',y_raw)

X_raw = X_raw.astype(np.float)

In [5]:
print(X_raw.shape)
print(y_raw.shape)

(86079, 342)
(86079, 48)


注意到y_raw有48行，他是对应的48小时我们要predict的数据，不能直接放到model去train，需要onehot要预测的是后几个小时
X_raw已经含有前三周的一些特征，包括最后一周的原始数据，三周内每一天的statistical 值[min max mean median var],每一周statistical的值,holiday feature 的值->[是否放假第一天，是否放假前一天，是否放假最后一天，是否上班第一天]


In [6]:
#这是用来选择部分站点的，因为feature前几行是站点的one_hot信息
def select_part_of_stations(train_x,train_y,station_num):
    x_list = []
    y_list = []
    for i in range(station_num):
        station_x = train_x[np.where(train_x[:,i] == 1)]
        station_y = train_y[np.where(train_x[:,i] == 1)].reshape(-1,1)
        x_list.append(station_x)
        y_list.append(station_y)
    return np.vstack(x_list),np.vstack(y_list)

In [ ]:
#把X_raw,y_ra 变成可以train的数据： one hot要preddict的value是48小时的哪一个(用48行的diag matrix), 复制48次X_raw 的每一行，把y_raw transpose
#一下变成48行

#我暂时内存不够，不能一次调用全部，只能拿部分站点出来做调参
train_2017_2018 = get_train_data_final(X_raw,y_raw)
train_x = train_data_2017_2018[:,:-1]
train_y = train_data_2017_2018[:,-1]